# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [419]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    # p - массив из долей каждого из классов
    def __gini_value(self, p):
        return 1 - np.sum((p ** 2))
    
    def __entropy_value(self, p):
        return -np.sum([p_k * np.log(p_k) for p_k in p])
    
    def __misclass_value(self, p):
        return 1 - np.max(p)
    
    
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        num_borders = l_c.shape[0]
        num_samples = l_s[0][0] + r_s[0][0]

        I_S = self.__gini_value((l_c[0] + r_c[0]) / num_samples)

        l_prob = l_c / l_s
        r_prob = r_c / r_s
        
        left = l_s * (1 - np.sum(l_prob ** 2, axis=1)).reshape(-1, 1)
        right = r_s * (1 - np.sum(r_prob ** 2, axis=1)).reshape(-1, 1)
        
        gs = I_S - (left + right)
        
        return gs
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        num_borders = l_c.shape[0]
        num_samples = l_s[0][0] + r_s[0][0]

        I_S = self.__entropy_value((l_c[0] + r_c[0]) / num_samples)

        l_prob = l_c / l_s
        r_prob = r_c / r_s 
        
        left = l_s * (-np.sum(l_prob * np.log(l_prob), axis=1)).reshape(-1, 1)
        right = r_s * (-np.sum(r_prob * np.log(l_prob), axis=1)).reshape(-1, 1)
        
        gs = I_S - (left + right)
        
        return gs

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        num_borders = l_c.shape[0]
        num_samples = l_s[0][0] + r_s[0][0]

        I_S = self.__misclass_value((l_c[0] + r_c[0]) / num_samples)

        l_prob = l_c / l_s
        r_prob = r_c / r_s 
        
        left = l_s * (1 - np.max(l_prob, axis=1)).reshape(-1, 1)
        right = r_s * (1 - np.max(r_prob, axis=1)).reshape(-1, 1)
        
        gs = I_S - (left + right)
        
        return gs
        
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        num_features = max(1, int(np.sqrt(n_feature)))
        return np.array(feature_ids[:num_features])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        num_features = max(1, int(np.log2(n_feature)))
        return np.array(feature_ids[:num_features])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # x - вектор значений для выбранного признака, y - значение целевой переменной
        # сортируем x и y в порядке неубывания значений x
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0] # число классов (различные значения целевой переменной)
        
        # Что делает этот блок кода?
        # мы ищем те индексы y, где значение целевой переменной меняет свой класс (но нам не интересуют разбиения,
        # при которых в одном из сплитов будет меньше объектов чем min_samples_split)
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # находим кол-во одинаковых подряд идущих значений классов между каджой парой индексов смены целевой переменной
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        
        # создаем матрицу где для каждого индекса границ разбиения будет сопоставлен one-hot вектор с единицей 
        # в позиции класса, котрый меняется после границы
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        
        # домножаем каждую строку (соответствующую границе) на кол-во подряд идущих элементов класса до смены на границе
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        
        # для значений классов до первой границы добавляем соответствующие значения
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # для кадого класса считаем сколько значений этого класса будет слева от первого, второго и т.д. разбиения 
        # (не включая значение класса по индексу самого разбиения)
        l_class_count = np.cumsum(class_increments, axis=0)  
        
        # то же самое только для дугой стороны (смотрим кол-во значений класса от границы включительно до конца)
        r_class_count = np.bincount(y) - l_class_count
        
        # считает кол-во элементов до кажлой из границ
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
       
        # считаем кол-во элементов после каждой из границ, включая границу
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считает значения критерия gain для разбиений 
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # ищем индекс наилучшего (минимального) значения 
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # находим индекс начала разбиения и вычисляем значение порога как среднее значение между элементами 
        # слева и справа от границы
        # возвращаем значение gain и threshhold
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE
        
        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split
        
        # Если класс единственный
        class_number = np.unique(y).shape[0]
        if class_number == 0:
            self.tree[node_id] = tuple((self.LEAF_TYPE, int(y[0]), 0))
            return
        
        # считаем кол-во элементов каждого класса
        class_dict = dict()
        for k in y: 
            if int(k) not in class_dict:
                class_dict[int(k)] = 1
            else:
                class_dict[int(k)] += 1
        
        class_items = class_dict.items()
        class_items.sort(key=lambda x: x[1], reverse=True)
        max_freq_c, num_max_freq_c = class_items[0]
                
        # ограничение на число элементов одинакового класса в узле
        if (num_max_freq_c / y.shape[0] >= self.sufficient_share):
            self.tree[node_id] = tuple((self.LEAF_TYPE, int(max_freq_c), None))
            return
        
        # если достигли максимальной глубины или кол-во объектов в вершине меньше минимального кол-ва для разбиения
        if depth == self.max_depth or x.shape[0] < self.min_samples_split:
            self.tree[node_id] = (self.LEAF_TYPE, int(max_freq_c), None)
            return
        
        feature_ids = self.get_feature_ids(x.shape[1])

        
        # в каждой строке будет feature_id, gain, threshold
        gain_thresh = np.zeros((feature_ids.shape[0], 3))
        for idx in xrange(feature_ids.shape[0]):
            gain_thresh[idx][0] = feature_ids[idx]
            gain_thresh[idx][1], gain_thresh[idx][2] = self.__find_threshold(x[:,idx], y)
            
        # сортируем по значению gain
        min_gain_index = np.argmin(gain_thresh[:,1], axis=0)
 
        self.tree[node_id] = tuple((self.NON_LEAF_TYPE, 
                                    int(gain_thresh[min_gain_index, 0]), 
                                    gain_thresh[min_gain_index, 2]))
        
        node = self.tree[node_id]
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, node[0], node[2])
        
        if y_left.shape[0] != 0 and y_right.shape[0] != 0:
            self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)    
        else:
            # получилось разбиение на два класса, в одном из которых 0 элементов, 
            self.tree[node_id] = tuple((self.LEAF_TYPE, int(max_freq_c), 0))

    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [420]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_features='sqrt', criterion='misclass')
my_clf.fit(x, y)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [5]:
df.shape

(120269, 11)

In [409]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [410]:
x.shape

(120269, 10)

In [411]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [412]:
my_clf.fit(x, y)

## Проверка скорости работы

In [421]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.43058681488
1.22270011902


## Проверка качества работы

In [414]:
gkf = KFold(n_splits=5, shuffle=True)

In [415]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.933690862227
0.92886837948
0.928411075081
0.931986364014
0.929613769592


In [416]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.892450320113
0.891993015715
0.894944707741
0.892076161969
0.889535608864
